In [1]:
## Lets import our requirements
import boto3
import pandas as pd
#import psycopg2
import pymysql
from sqlalchemy import create_engine
import os
import sqlite3
import sys
parentDirectory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0,parentDirectory)
from secretsPaul import secrets
import mysql.connector


     ---------------------------------------- 11.9/11.9 MB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp39-cp39-win_amd64.whl size=247946 sha256=cfadce99788a5b7d28ac1a79f9c285e83b95b88195e0922ba048392364f5fe31
  Stored in directory: c:\users\paull\appdata\local\pip\cache\wheels\7b\14\39\5aad423666e827dfe9a1fbcd111ac17171e7c9865d570780ce
Successfully built mysql.connector
Note: you may need to restart the kernel to use updated packages.


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [11]:
#connections
endpoint = secrets.get('DATABASE_ENDPOINT')

user = secrets.get('DATABASE_USER')

password = secrets.get('DATABASE_PASSWORD')

port=secrets.get('DATABASE_PORT')

connection = create_engine(f'mysql+pymysql://{user}:{password}@{endpoint}:{port}/Capstone', pool_recycle=3600);

connect=mysql.connector.connect(host=endpoint, database='Capstone', user=user, password=password)
cursor = connect.cursor()


In [12]:
listing_detail = pd.read_csv(os.path.join(parentDirectory, 'listings.csv'))
listing_detail.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,108061,https://www.airbnb.com/rooms/108061,20221221041414,2022-12-21,city scrape,Walk to stores/parks/downtown. Fenced yard/Pet...,Walk to town in ten minutes! Monthly rental $1...,"I love my neighborhood! Its friendly, easy-goi...",https://a0.muscache.com/pictures/miso/Hosting-...,320564,https://www.airbnb.com/users/show/320564,Lisa,2010-12-16,"Asheville, NC",I am a long time resident of Asheville and am ...,within a few hours,100%,64%,f,https://a0.muscache.com/im/users/320564/profil...,https://a0.muscache.com/im/users/320564/profil...,NaN,2,3,"['email', 'phone']",t,t,"Asheville, North Carolina, United States",28801,NaN,35.60670,-82.55563,Entire rental unit,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Free street parking"", ""Cleaning products"", ""...",$100.00,10,365,10,10,1125,1125,10.0,1125.0,NaN,t,11,11,11,180,2022-12-21,91,1,0,2011-09-21,2022-10-08,4.51,4.57,4.70,4.85,4.80,4.84,4.48,NaN,f,2,2,0,0,0.66
1,155305,https://www.airbnb.com/rooms/155305,20221221041414,2022-12-21,city scrape,Cottage! BonPaul + Sharky's Hostel,<b>The space</b><br />Pet friendly private cot...,"We are within easy walk of pubs, breweries, mu...",https://a0.muscache.com/pictures/8880711/cf38d...,746673,https://www.airbnb.com/users/show/746673,BonPaul,2011-06-26,"Asheville, NC",We operate two traveler's hostels located in H...,within an hour,90%,98%,t,https://a0.muscache.com/im/pictures/user/4dff7...,https://a0.muscache.com/im/pictures/user/4dff7...,NaN,7,11,"['email', 'phone']",t,t,"Asheville, North Carolina, United States",28806,NaN,35.57864,-82.59578,Entire guesthouse,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Free street parking"", ""Oven"", ""Long term sta...",$100.00,1,365,1,1,7,365,1.0,55.6,NaN,t,19,49,79,300,2022-12-21,374,27,1,2011-07-31,2022-12-04,4.58,4.71,4.41,4.83,4.74,4.93,4.54,NaN,f,7,1,2,4,2.70
2,156805,https://www.airbnb.com/rooms/156805,20221221041414,2022-12-21,previous scrape,"Private Room ""Ader"" at BPS Hostel",<b>The space</b><br />Private Rooms at Bon Pau...,"Easy walk to pubs, cafes, bakery, breweries, l...",https://a0.muscache.com/pictures/23447d55-fa7e...,746673,https://www.airbnb.com/users/show/746673,BonPaul,2011-06-26,"Asheville, NC",We operate two traveler's hostels located in H...,within an hour,90%,98%,t,https://a0.muscache.com/im/pictures/user/4dff7...,https://a0.muscache.com/im/pictures/user/4dff7...,NaN,7,11,"['email', 'phone']",t,t,"Asheville, North Carolina, United States",28806,NaN,35.57864,-82.59578,Private room in home,Private room,2,NaN,2.5 shared baths,1.0,1.0,"[""Free street parking"", ""Long term stays allow...",$66.00,1,365,1,1,365,365,1.0,365.0,NaN,t,0,0,0,0,2022-12-21,67,0,0,2011-09-20,2020-01-01,4.52,4.73,4.

In [13]:
listing_detail.rename(columns={'id':'listing_id', 'name':'listing_name'}, inplace=True)

In [14]:
listing_detail['price']=listing_detail['price'].str[1:].str.replace(',','').apply(pd.to_numeric) #changing price column to numeric field
listing_detail['host_acceptance_rate']=listing_detail['host_acceptance_rate'].str[:-1].apply(pd.to_numeric)/100 #change to numeric
listing_detail['host_response_rate']=listing_detail['host_response_rate'].str[:-1].apply(pd.to_numeric)/100

In [15]:
#converting to datetime
listing_detail[['last_scraped', 'host_since','calendar_last_scraped','first_review','last_review']]=listing_detail[['last_scraped', 'host_since','calendar_last_scraped','first_review','last_review']].apply(pd.to_datetime)

In [16]:
#converting to 1/0 (better for modeling and summarization)
listing_detail[['host_is_superhost','host_has_profile_pic','host_identity_verified','has_availability','instant_bookable']]=listing_detail[['host_is_superhost','host_has_profile_pic','host_identity_verified','has_availability','instant_bookable']].replace('t',1).replace('f',0)

In [17]:
listing_detail.to_sql('listing_detail_stage',con=connection, if_exists='replace',index=False)

3093

### Calendar

In [18]:
calendar = pd.read_csv(os.path.join(parentDirectory, 'calendar.csv'))
calendar.head()


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,108061,2022-12-21,t,$100.00,$100.00,10.0,1125.0
1,108061,2022-12-22,t,$100.00,$100.00,10.0,1125.0
2,108061,2022-12-23,t,$100.00,$100.00,10.0,1125.0
3,108061,2022-12-24,t,$100.00,$100.00,10.0,1125.0
4,108061,2022-12-25,t,$100.00,$100.00,10.0,1125.0


In [19]:
calendar['price']=pd.to_numeric(calendar['price'].str[1:].str.replace(',','')) #changing price column to numeric field
calendar['adjusted_price']=pd.to_numeric(calendar['adjusted_price'].str[1:].str.replace(',','')) #changing price column to numeric field
calendar['date']=pd.to_datetime(calendar['date'],infer_datetime_format=True)
calendar['available']=calendar['available'].replace('t',1).replace('f',0)

In [20]:
calendar.rename(columns={'date':'calendar_date'}, inplace=True)

In [8]:
from sqlalchemy import event
@event.listens_for(connection, "before_cursor_execute")
def receive_before_cursor_execute(
       conn, cursor, statement, params, context, executemany
        ):
            if executemany:
                cursor.fast_executemany = True

In [ ]:
calendar.to_sql('calendar_stage',con=connection, if_exists='replace',index=False, method='multi')

In [29]:
calendar.to_csv('data.csv')

In [21]:
cursor.execute("select database();")
record=cursor.fetchone()
print(record)

('Capstone',)


In [10]:
sql_query = "LOAD DATA LOCAL INFILE 'data.csv' REPLACE INTO TABLE calendar_stage FIELDS TERMINATED BY ',';"
%timeit -r 1 -n 1 cursor.execute(sql_query)

46.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Reviews


In [22]:
reviews = pd.read_csv(os.path.join(parentDirectory, 'reviews.csv'))
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,108061,553741,2011-09-21,822907,Pedro & Katie,"Lisa is superb hostess, she will treat you lik..."
1,108061,683278,2011-11-01,236064,Tim,This was a lovely little place walking distanc...
2,108061,714889,2011-11-13,1382707,Shane,"Lisa was very nice to work with. However, we ..."
3,108061,1766157,2012-07-21,416731,Brenda,I feel very lucky to have found this beautiful...
4,108061,2033065,2012-08-19,1858880,Lindsey,"Great roomy little apartment, beautiful privat..."


In [23]:
reviews['date']=reviews['date'].apply(pd.to_datetime)

In [24]:
reviews.rename(columns={'date':'review_date'}, inplace=True)

In [25]:
reviews.to_sql('reviews_stage',con=connection, if_exists='replace',index=False)

294517